### Before the start

In [ ]:
# # before start
# %pip install tld
# %pip install category_encoders

# ! pip install pytz
# !pip install timezonefinder
# !pip install geopy

# !pip3 install torch
# !pip3 install torchvision
# !pip3 install tqdm

# !pip install torch -q
# !pip install keras

# !pip install skorch
# !pip install bayesian-optimization

# # !pip install cudf
# # !pip install cudf-cu11==22.12 rmm-cu11==22.12 dask-cudf-cu11==22.12 --extra-index-url=https://pypi.ngc.nvidia.com/

In [ ]:
# load data and import libraries
import pandas as pd
import numpy as np
import pytz
# from geopy.geocoders import Nominatim
# from timezonefinder import TimezoneFinder
import datetime

import torch
# For layers and models
import torch.nn as nn
# For activation functions (F.sigmoid is deprecated)
import torch.nn.functional as F
from torch import sigmoid

import os
import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import Dataset, DataLoader
# Collection of optimizers
import torch.optim as optim

# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
# model = CustomModel().to(device)

### ANN preprocessing

In [ ]:
new_labels = [
    "Sport",
    "Hobby Passive",
    "Work&Study",
    "Daily Routine"
]
match_labels_v3 = [0,3,0,1,2,2,3]

match_labels = {
    "Sport": 0,
    "Food": 3,
    "Hobby Active": 0,
    "Hobby Passive": 1,
    "Studying":2,
    "Work":2,
    "Other":3
}


In [ ]:
df = pd.read_csv('schedule.csv')

df['Label Number'] = df['Label'].apply(lambda x: match_labels[x])
df['Label Name'] = df['Label Number'].apply(lambda x: new_labels[x])

df.drop(columns=['Label', 'Activity', 'End Time', 'If Has Deadline', 'Deadline Time', 'Deadline Date'], inplace=True)

mini = df['Importance'].min()
maxi = df['Importance'].max()
df['Importance'] = df['Importance'].apply(lambda x: int((x-mini) / (maxi-mini) * 3) )

df[:13]

,Date,Start Time,Duration,Importance,Label Number,Label Name
0,2023-06-16,07:00,60,1,3,Daily Routine
1,2023-06-16,08:15,45,2,0,Sport
2,2023-06-16,09:15,60,0,1,Hobby Passive
3,2023-06-16,10:30,45,3,2,Work&Study
4,2023-06-16,11:30,60,1,3,Daily Routine
5,2023-06-16,13:00,60,2,2,Work&Study
6,2023-06-16,14:15,30,0,0,Sport
7,2023-06-16,15:00,90,1,2,Work&Study
8,2023-06-16,17:00,45,1,3,Daily Routine
9,2023-06-16,18:00,45,2,0,Sport


In [ ]:
from sklearn.preprocessing import OneHotEncoder

def encode_features(df, features_name, encoder):
    new_features = encoder.transform(df[features_name])
    # create dataframe from encoded features with named columns
    new_columns = pd.DataFrame(new_features, columns=encoder.get_feature_names_out())
    new_df = pd.concat([df, new_columns], axis=1)
    # new_df.drop(features_name, axis=1, inplace=True)
    return new_df

encoder = OneHotEncoder(sparse=False, drop='first')
enc_column = ['Label Number']
encoder.fit(df[enc_column])
df = encode_features(df, enc_column, encoder)

df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3
0,2023-06-16,07:00,60,1,3,Daily Routine,0.0,0.0,1.0
1,2023-06-16,08:15,45,2,0,Sport,0.0,0.0,0.0
2,2023-06-16,09:15,60,0,1,Hobby Passive,1.0,0.0,0.0
3,2023-06-16,10:30,45,3,2,Work&Study,0.0,1.0,0.0
4,2023-06-16,11:30,60,1,3,Daily Routine,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
249,2023-07-07,15:45,90,2,2,Work&Study,0.0,1.0,0.0
250,2023-07-07,17:30,45,1,3,Daily Routine,0.0,0.0,1.0
251,2023-07-07,18:30,90,1,0,Sport,0.0,0.0,0.0
252,2023-07-07,20:15,90,2,3,Daily Routine,0.0,0.0,1.0


In [ ]:
def make_true_duration(x, prob=1.3):
    true_noise = [[-5,20], [-15,0], [-20,5], [-10,30]]
    addi = np.random.uniform(low=prob*true_noise[x['Label Number']][0], high=prob * true_noise[x['Label Number']][1])
    result= x['Duration'] + int(addi)
    if addi > true_noise[x['Label Number']][1]:
        result= x['Duration'] + true_noise[x['Label Number']][1]
    elif addi < true_noise[x['Label Number']][0]:
        result= x['Duration'] + true_noise[x['Label Number']][0]
    if result < 10:
        return 10
    return result

df['True Duration'] = df.apply(lambda x: make_true_duration(x), axis=1)
df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3,True Duration
0,2023-06-16,07:00,60,1,3,Daily Routine,0.0,0.0,1.0,90
1,2023-06-16,08:15,45,2,0,Sport,0.0,0.0,0.0,64
2,2023-06-16,09:15,60,0,1,Hobby Passive,1.0,0.0,0.0,45
3,2023-06-16,10:30,45,3,2,Work&Study,0.0,1.0,0.0,25
4,2023-06-16,11:30,60,1,3,Daily Routine,0.0,0.0,1.0,77
...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,15:45,90,2,2,Work&Study,0.0,1.0,0.0,71
250,2023-07-07,17:30,45,1,3,Daily Routine,0.0,0.0,1.0,57
251,2023-07-07,18:30,90,1,0,Sport,0.0,0.0,0.0,97
252,2023-07-07,20:15,90,2,3,Daily Routine,0.0,0.0,1.0,103


In [ ]:
min_time = datetime.datetime.strptime("00:00", '%H:%M')
max_time = ((datetime.datetime.strptime("23:59", '%H:%M')-min_time).total_seconds()+1)//60
# df['Start Time'] = df['Start Time'].apply(lambda x: ((datetime.datetime.strptime(x, '%H:%M')-min_time).total_seconds()+1)/max_time)
df['Start Time'] = df['Start Time'].apply(lambda x: (datetime.datetime.strptime(x, '%H:%M')-min_time).total_seconds()//60)

# move start time
cur_day = '1/1/2020'
delta = 0

for index, row in df.iterrows():
    if row['Date'] != cur_day:
        delta = row['Start Time']
        cur_day = row['Date']
    df.at[index, 'Start Time'] -= delta

df['True Start Time'] = df['Start Time']
df.describe()

,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000
mean,426.496063,66.259843,1.251969,1.779528,0.110236,0.220472,0.409449,70.877953,426.496063
std,278.493487,19.999689,0.897678,1.231429,0.313802,0.415384,0.492703,23.796836,278.493487
min,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000
25%,180.000000,45.000000,1.000000,0.000000,0.000000,0.000000,0.000000,52.000000,180.000000
50%,420.000000,60.000000,1.000000,2.000000,0.000000,0.000000,0.000000,70.500000,420.000000
75%,660.000000,90.000000,2.000000,3.000000,0.000000,0.000000,1.000000,90.000000,660.000000
max,900.000000,90.000000,3.000000,3.000000,1.000000,1.000000,1.000000,120.000000,900.000000


In [ ]:
TS_DUR =10
LENGTH = 1200

def parse_date(d):
    res = [int(i) for i in d.split('/')[::-1]]
    return datetime.date(*res)

cur_day = '1/1/2020'
# basic_slots = [0]*(2*60//10)+[1]*(3*60//10)+[0]*(2*60//10)
basic_slots = [0]*(LENGTH//TS_DUR)

available_time_slots = [0]*(LENGTH//TS_DUR)
names = ['TS'+str(i) for i in range(len(available_time_slots))]
df[names] = available_time_slots

for index, row in df.iterrows():
    if row['Date'] == cur_day:
        df.loc[index, names] = available_time_slots.copy()
    else:
        cur_day = row['Date']
        available_time_slots = basic_slots.copy()
        df.loc[index, names] = available_time_slots.copy()
    start = int(row['True Start Time'] / TS_DUR)
    end = int((row['True Start Time'] + row['True Duration']) // TS_DUR) + [1 if (row['True Start Time'] + row['True Duration']) % TS_DUR > 0 else 0][0]
    # print(start, end, row['True Duration'] )
    df.loc[index, 'Summ'] = sum(available_time_slots)

    available_time_slots[start:end+1] = [1]*(end-start+1)
    # print(index, available_time_slots)

df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3,True Duration,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.0,60,1,3,Daily Routine,0.0,0.0,1.0,90,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,75.0,45,2,0,Sport,0.0,0.0,0.0,64,...,0,0,0,0,0,0,0,0,0,10.0
2,2023-06-16,135.0,60,0,1,Hobby Passive,1.0,0.0,0.0,45,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,210.0,45,3,2,Work&Study,0.0,1.0,0.0,25,...,0,0,0,0,0,0,0,0,0,19.0
4,2023-06-16,270.0,60,1,3,Daily Routine,0.0,0.0,1.0,77,...,0,0,0,0,0,0,0,0,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,525.0,90,2,2,Work&Study,0.0,1.0,0.0,71,...,0,0,0,0,0,0,0,0,0,53.0
250,2023-07-07,630.0,45,1,3,Daily Routine,0.0,0.0,1.0,57,...,0,0,0,0,0,0,0,0,0,60.0
251,2023-07-07,690.0,90,1,0,Sport,0.0,0.0,0.0,97,...,0,0,0,0,0,0,0,0,0,67.0
252,2023-07-07,795.0,90,2,3,Daily Routine,0.0,0.0,1.0,103,...,0,0,0,0,0,0,0,0,0,77.0


In [ ]:
# scaling
df['Importance'] = df['Importance'] / 3

df.drop(columns=['Label Name'], inplace=True)

df['Duration'] = df['Duration'] / (6*60)
df['True Duration'] = df['True Duration'] / (6*60)

df

,Date,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.0,0.166667,0.333333,3,0.0,0.0,1.0,0.250000,0.0,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,75.0,0.125000,0.666667,0,0.0,0.0,0.0,0.177778,75.0,...,0,0,0,0,0,0,0,0,0,10.0
2,2023-06-16,135.0,0.166667,0.000000,1,1.0,0.0,0.0,0.125000,135.0,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,210.0,0.125000,1.000000,2,0.0,1.0,0.0,0.069444,210.0,...,0,0,0,0,0,0,0,0,0,19.0
4,2023-06-16,270.0,0.166667,0.333333,3,0.0,0.0,1.0,0.213889,270.0,...,0,0,0,0,0,0,0,0,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,525.0,0.250000,0.666667,2,0.0,1.0,0.0,0.197222,525.0,...,0,0,0,0,0,0,0,0,0,53.0
250,2023-07-07,630.0,0.125000,0.333333,3,0.0,0.0,1.0,0.158333,630.0,...,0,0,0,0,0,0,0,0,0,60.0
251,2023-07-07,690.0,0.250000,0.333333,0,0.0,0.0,0.0,0.269444,690.0,...,0,0,0,0,0,0,0,0,0,67.0
252,2023-07-07,795.0,0.250000,0.666667,3,0.0,0.0,1.0,0.286111,795.0,...,0,0,0,0,0,0,0,0,0,77.0


In [ ]:
def drop_some_values(x, prob=0.95 ):
    if np.random.uniform() > prob:
        return 0
    return x

df['Duration'] = df['Duration'].apply(lambda x: drop_some_values(x))

df['Start Time'] = df['Start Time'] / max_time
df['True Start Time'] = df['Start Time']
df['Start Time'] = df['Start Time'].apply(lambda x: drop_some_values(x))
df.describe()

,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,TS0,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,...,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.000000
mean,0.281240,0.173228,0.417323,1.779528,0.110236,0.220472,0.409449,0.196883,0.296384,0.913386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.767717
std,0.198370,0.068295,0.299226,1.231429,0.313802,0.415384,0.492703,0.066102,0.193533,0.281824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.283292
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.093815,0.125000,0.333333,0.000000,0.000000,0.000000,0.000000,0.144444,0.125087,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.250000
50%,0.260598,0.166667,0.333333,2.000000,0.000000,0.000000,0.000000,0.195833,0.291869,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.000000
75%,0.458652,0.250000,0.666667,3.000000,0.000000,0.000000,1.000000,0.250000,0.458652,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.000000
max,0.625434,0.250000,1.000000,3.000000,1.000000,1.000000,1.000000,0.333333,0.625434,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.000000


In [ ]:
df

,Date,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.000000,0.166667,0.333333,3,0.0,0.0,1.0,0.250000,0.000000,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,0.052120,0.125000,0.666667,0,0.0,0.0,0.0,0.177778,0.052120,...,0,0,0,0,0,0,0,0,0,10.0
2,2023-06-16,0.093815,0.166667,0.000000,1,1.0,0.0,0.0,0.125000,0.093815,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,0.145935,0.125000,1.000000,2,0.0,1.0,0.0,0.069444,0.145935,...,0,0,0,0,0,0,0,0,0,19.0
4,2023-06-16,0.187630,0.166667,0.333333,3,0.0,0.0,1.0,0.213889,0.187630,...,0,0,0,0,0,0,0,0,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,0.364837,0.000000,0.666667,2,0.0,1.0,0.0,0.197222,0.364837,...,0,0,0,0,0,0,0,0,0,53.0
250,2023-07-07,0.437804,0.125000,0.333333,3,0.0,0.0,1.0,0.158333,0.437804,...,0,0,0,0,0,0,0,0,0,60.0
251,2023-07-07,0.479500,0.250000,0.333333,0,0.0,0.0,0.0,0.269444,0.479500,...,0,0,0,0,0,0,0,0,0,67.0
252,2023-07-07,0.552467,0.250000,0.666667,3,0.0,0.0,1.0,0.286111,0.552467,...,0,0,0,0,0,0,0,0,0,77.0


### Test train dataset

In [ ]:
X = df.drop(columns=['Date', 'True Duration', 'True Start Time', 'Label Number', 'Summ'])
y = df[['True Start Time', 'True Duration']]

In [ ]:
len(X.columns)

126

In [ ]:
y.describe()

,True Start Time,True Duration
count,254.000000,254.000000
mean,0.296384,0.196883
std,0.193533,0.066102
min,0.000000,0.041667
25%,0.125087,0.144444
50%,0.291869,0.195833
75%,0.458652,0.250000
max,0.625434,0.333333


In [ ]:
X = X.to_numpy()
y = y.to_numpy()


In [ ]:
y

array([[0.        , 0.25      ],
       [0.05211953, 0.17777778],
       [0.09381515, 0.125     ],
       [0.14593468, 0.06944444],
       [0.1876303 , 0.21388889],
       [0.25017373, 0.11666667],
       [0.30229326, 0.09444444],
       [0.33356498, 0.21388889],
       [0.41695622, 0.10833333],
       [0.45865184, 0.13888889],
       [0.50034746, 0.125     ],
       [0.55246699, 0.04166667],
       [0.58373871, 0.22777778],
       [0.        , 0.225     ],
       [0.05211953, 0.17777778],
       [0.09381515, 0.19444444],
       [0.16678249, 0.10555556],
       [0.20847811, 0.11944444],
       [0.26059764, 0.27777778],
       [0.33356498, 0.19444444],
       [0.41695622, 0.16666667],
       [0.45865184, 0.24722222],
       [0.53161918, 0.31388889],
       [0.60458652, 0.20277778],
       [0.        , 0.14166667],
       [0.05211953, 0.30555556],
       [0.12508687, 0.125     ],
       [0.17720639, 0.11944444],
       [0.21890202, 0.25      ],
       [0.29186935, 0.3       ],
       [0.

In [ ]:
len(torch.tensor(y,dtype=torch.float32))

254

In [ ]:
from sklearn.model_selection import train_test_split

class TaskDataset(Dataset):
  def __init__(self,X, y):
    self.x_train=torch.tensor(X,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self,idx):
    return self.x_train[idx], self.y_train[idx]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
datasets = {
    'train': TaskDataset(X_train, y_train),
    'test': TaskDataset(X_test, y_test)
}

batch_size = 10
dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=batch_size, shuffle=False),
    # 'val': DataLoader(valData, batch_size=batch_size, shuffle=False),
    'test': DataLoader(datasets['test'], batch_size=batch_size, shuffle=False)
}

### Model designing

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
# Definition of the very simple network with 1 hidden layer
class SimpleScheduler(nn.Module):
    def __init__(self, input_n=126):
        # Invoke the parent's constructor
        super(SimpleScheduler, self).__init__()
        # nn implements feedworward layer as nn.Linear(a, b),
        # where a, b - input and output dims of weight matrix.
        # Bias is included by default.
        self.hidden = nn.Linear(input_n, 50)
        self.output = nn.Linear(50, 2)


    def forward(self, x):
        # You can write the result of sigmoid to a local variable,
        # but the common style in torch is to re-write x after computation of each layer
        x = F.relu(self.hidden(x))
        x = self.output(x)
        return x


model1 = SimpleScheduler().to(device)

print(f"SimpleScheduler model architecture:\n {model1}")
print("\n")

SimpleScheduler model architecture:
 SimpleScheduler(
  (hidden): Linear(in_features=126, out_features=50, bias=True)
  (output): Linear(in_features=50, out_features=2, bias=True)
)




In [ ]:
# Set hyperparameters
# Number of iterations over the whole data set
epochs = 50
# Learning rate for Stochastic Gradient Descent
lr = 0.005
# SGD parameter to accelerate the optimization, check https://paperswithcode.com/method/sgd-with-momentum
momentum = 0.5
# Loss function - cross entropy, the multiclass variant of that you used in Lab 3
criterion = nn.HuberLoss()

In [ ]:
from tqdm import tqdm

def train( model, device, train_loader, optimizer, epoch):
    # Do not forget to set train() to update weights of model
    model.train()
    # A wrapper over data loader to show progress bar
    bar = tqdm(train_loader)
    iteration = 0
    overall_loss = 0
    for data, target in bar:
        # Device of data and model must be the same
        data, target = data.to(device), target.to(device)
        # To avoid an accumulation of gradients
        optimizer.zero_grad()
        # Prediction
        output = model(data)
        # Error between prediction and ground truth
        loss = criterion(output, target)
        # Compute gradient
        loss.backward()
        # Update params of model
        optimizer.step()

        iteration += 1
        overall_loss += loss.item()
        bar.set_postfix({"Loss": format(overall_loss/iteration, '.6f')})

def test( model, device, test_loader):
    # Do not forget to set eval() to keep model's params the same
    model.eval()
    test_loss = 0
    correct = 0
    mse_loss = nn.MSELoss()
    mse_loss_val = 0
    ind = 0
    with torch.no_grad():
        for data, target in test_loader:
            # print(f"target shape is {target.shape}")
            data, target = data.to(device), target.to(device)
            output = model(data)
            # print(f"output shape is {output.shape}")
            # Sum up batch loss
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            mse_loss_val += mse_loss(output, target).item()
            # mse_loss += sum()
            # pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # print(output.shape)
            # targ = target.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            print(target, "\n", output)

            # correct += pred.eq(targ.view_as(pred)).sum().item()

    # test_loss /= len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss} mse loss: {mse_loss_val}")

In [ ]:
# Train the model
optimizer = optim.Adam(model1.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model1, device, dataloaders['train'], optimizer, epoch)
    test(model1, device, dataloaders['test'])

# Optionally, you can save the params of model to file
#torch.save(model.state_dict(), "mnist_model.pt")


100%|██████████| 21/21 [00:00<00:00, 215.47it/s, Loss=0.003354]


tensor([[0.3648, 0.2528],
        [0.4378, 0.1389],
        [0.4795, 0.2806],
        [0.5525, 0.3056],
        [0.6254, 0.2194],
        [0.0000, 0.1778],
        [0.0521, 0.1806],
        [0.0938, 0.1500],
        [0.1459, 0.0694],
        [0.1876, 0.2500]]) 
 tensor([[0.3791, 0.1596],
        [0.4953, 0.1324],
        [0.5037, 0.1630],
        [0.6091, 0.1351],
        [0.6914, 0.0640],
        [0.0026, 0.2019],
        [0.0525, 0.1969],
        [0.1308, 0.0937],
        [0.1718, 0.1208],
        [0.2052, 0.1265]])
tensor([[0.2502, 0.1444],
        [0.3023, 0.0833],
        [0.3336, 0.1944],
        [0.4170, 0.1972],
        [0.4587, 0.1278],
        [0.5003, 0.1500],
        [0.5525, 0.0611],
        [0.5837, 0.1556],
        [0.0000, 0.2389],
        [0.0521, 0.2889]]) 
 tensor([[0.2828, 0.1673],
        [0.2977, 0.1113],
        [0.3755, 0.1140],
        [0.4272, 0.1177],
        [0.4661, 0.1406],
        [0.4721, 0.1208],
        [0.5234, 0.0963],
        [0.5896, 0.0921],
     

100%|██████████| 21/21 [00:00<00:00, 285.34it/s, Loss=0.001241]


tensor([[0.3648, 0.2528],
        [0.4378, 0.1389],
        [0.4795, 0.2806],
        [0.5525, 0.3056],
        [0.6254, 0.2194],
        [0.0000, 0.1778],
        [0.0521, 0.1806],
        [0.0938, 0.1500],
        [0.1459, 0.0694],
        [0.1876, 0.2500]]) 
 tensor([[0.3361, 0.1960],
        [0.4363, 0.1762],
        [0.4540, 0.2120],
        [0.5637, 0.2341],
        [0.6037, 0.1845],
        [0.0017, 0.1907],
        [0.0368, 0.2030],
        [0.0891, 0.1256],
        [0.1334, 0.1343],
        [0.1593, 0.1467]])
tensor([[0.2502, 0.1444],
        [0.3023, 0.0833],
        [0.3336, 0.1944],
        [0.4170, 0.1972],
        [0.4587, 0.1278],
        [0.5003, 0.1500],
        [0.5525, 0.0611],
        [0.5837, 0.1556],
        [0.0000, 0.2389],
        [0.0521, 0.2889]]) 
 tensor([[0.2153, 0.1829],
        [0.2285, 0.1551],
        [0.2934, 0.1691],
        [0.3350, 0.1353],
        [0.3905, 0.1863],
        [0.3899, 0.1741],
        [0.4520, 0.1617],
        [0.5145, 0.1842],
     

100%|██████████| 21/21 [00:00<00:00, 217.19it/s, Loss=0.000756]


...

tensor([[0.3648, 0.2528],
        [0.4378, 0.1389],
        [0.4795, 0.2806],
        [0.5525, 0.3056],
        [0.6254, 0.2194],
        [0.0000, 0.1778],
        [0.0521, 0.1806],
        [0.0938, 0.1500],
        [0.1459, 0.0694],
        [0.1876, 0.2500]]) 
 tensor([[0.3794, 0.2169],
        [0.4598, 0.1567],
        [0.4857, 0.2342],
        [0.5815, 0.2662],
        [0.6521, 0.1966],
        [0.0047, 0.1725],
        [0.0583, 0.1776],
        [0.0882, 0.1295],
        [0.1477, 0.1011],
        [0.1872, 0.1919]])
tensor([[0.2502, 0.1444],
        [0.3023, 0.0833],
        [0.3336, 0.1944],
        [0.4170, 0.1972],
        [0.4587, 0.1278],
        [0.5003, 0.1500],
        [0.5525, 0.0611],
        [0.5837, 0.1556],
        [0.0000, 0.2389],
        [0.0521, 0.2889]]) 
 tensor([[ 0.2464,  0.1028],
        [ 0.3301,  0.0672],
        [ 0.3503,  0.2271],
        [ 0.4360,  0.1316],
        [ 0.4861,  0.1703],
        [ 0.5194,  0.1402],
        [ 0.5735,  0.0418],
        [ 0.6075,

100%|██████████| 21/21 [00:00<00:00, 304.88it/s, Loss=0.000407]


tensor([[0.3648, 0.2528],
        [0.4378, 0.1389],
        [0.4795, 0.2806],
        [0.5525, 0.3056],
        [0.6254, 0.2194],
        [0.0000, 0.1778],
        [0.0521, 0.1806],
        [0.0938, 0.1500],
        [0.1459, 0.0694],
        [0.1876, 0.2500]]) 
 tensor([[0.4091, 0.2298],
        [0.4891, 0.1761],
        [0.5141, 0.2638],
        [0.6126, 0.2993],
        [0.6947, 0.2250],
        [0.0076, 0.1736],
        [0.0616, 0.1838],
        [0.0960, 0.1361],
        [0.1590, 0.1044],
        [0.1991, 0.1957]])
tensor([[0.2502, 0.1444],
        [0.3023, 0.0833],
        [0.3336, 0.1944],
        [0.4170, 0.1972],
        [0.4587, 0.1278],
        [0.5003, 0.1500],
        [0.5525, 0.0611],
        [0.5837, 0.1556],
        [0.0000, 0.2389],
        [0.0521, 0.2889]]) 
 tensor([[0.2733, 0.1158],
        [0.3532, 0.0817],
        [0.3711, 0.2391],
        [0.4535, 0.1488],
        [0.5064, 0.1940],
        [0.5425, 0.1624],
        [0.6006, 0.0635],
        [0.6296, 0.2049],
     

100%|██████████| 21/21 [00:00<00:00, 263.53it/s, Loss=0.000419]


tensor([[0.3648, 0.2528],
        [0.4378, 0.1389],
        [0.4795, 0.2806],
        [0.5525, 0.3056],
        [0.6254, 0.2194],
        [0.0000, 0.1778],
        [0.0521, 0.1806],
        [0.0938, 0.1500],
        [0.1459, 0.0694],
        [0.1876, 0.2500]]) 
 tensor([[0.3971, 0.2390],
        [0.4737, 0.1850],
        [0.4981, 0.2763],
        [0.5941, 0.3016],
        [0.6811, 0.2209],
        [0.0042, 0.1775],
        [0.0623, 0.1817],
        [0.0981, 0.1423],
        [0.1613, 0.1148],
        [0.1854, 0.2006]])
tensor([[0.2502, 0.1444],
        [0.3023, 0.0833],
        [0.3336, 0.1944],
        [0.4170, 0.1972],
        [0.4587, 0.1278],
        [0.5003, 0.1500],
        [0.5525, 0.0611],
        [0.5837, 0.1556],
        [0.0000, 0.2389],
        [0.0521, 0.2889]]) 
 tensor([[ 2.5943e-01,  1.5123e-01],
        [ 3.2419e-01,  1.2531e-01],
        [ 3.4903e-01,  2.4652e-01],
        [ 4.2929e-01,  1.6555e-01],
        [ 4.8279e-01,  1.9300e-01],
        [ 5.1644e-01,  1.6424e-01